# Installation

pip install -U keras-tuner

### Optional (Only if errors appear)

1. pip install tensorflow --upgrade --user
2. pip install keras --upgrade --user

In [1]:
#install keras tuner

!pip install -U keras-tuner

  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=5a17134dee552b213ec92561c6809d363829eadf422e4075364a0cf8b88b2aca
  Stored in directory: c:\users\udara vimukthi\appdata\local\pip\cache\wheels\f2\2e\6b\ce6f26f303aa955673b067602e7e388625406cc84db29f257b
Successfully built kt-legacy


In [2]:
!pip install tensorflow --upgrade --user

Requirement already up-to-date: tensorflow in c:\users\udara vimukthi\anaconda3\lib\site-packages (2.5.0)


In [3]:
!pip install keras --upgrade --user

Requirement already up-to-date: keras in c:\users\udara vimukthi\appdata\roaming\python\python38\site-packages (2.4.3)


In [4]:
import pandas as pd
import numpy as np

dataset=pd.read_csv('cardio_dataset.csv')
print(dataset.head())
dataset=dataset.values

   SEX  AGEIR   TC  HDL  SMOKE_  BPMED  DIAB_01  RISK
0    1     48  236   66       0      1        0   1.1
1    0     48  260   51       0      1        1   7.0
2    0     44  187   49       1      1        0   7.0
3    1     42  216   57       1      1        0   0.4
4    1     56  156   42       0      1        0   2.2


In [5]:
data=dataset[:,0:7]
target=dataset[:,7]

In [6]:
from sklearn.preprocessing import MinMaxScaler

target=np.reshape(target, (-1,1))

scaler_data = MinMaxScaler(feature_range=(0,1))
scaler_target = MinMaxScaler()

data_scaled=scaler_data.fit_transform(data)
target_scaled=scaler_target.fit_transform(target)

In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data_scaled, target_scaled,test_size=0.2)

In [8]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import numpy as np

from keras.optimizers import Adam,Adagrad,Adadelta

def build_model(parameters):
    
    model = Sequential()
    
    for i in range(parameters.Int('num_layers', 2, 20)):

        if(i==0):
            model.add(Dense(units=parameters.Int('#neurons layer' + str(i),min_value=32,max_value=512,step=32),
                            activation=parameters.Choice('activation_function '+str(i),['relu','sigmoid','tanh']),input_dim=7))
            
            model.add(Dropout(parameters.Choice('drop_prob 1',[0.2,0.3,0.4,0.5])))
        
        else:
            model.add(Dense(units=parameters.Int('#neurons layer' + str(i),min_value=32,max_value=512,step=32),
                                   activation=parameters.Choice('activation_function '+str(i),['relu','sigmoid','tanh'])))
            
            model.add(Dropout(parameters.Choice('drop_prob 2',[0.2,0.3,0.4,0.5])))
            
    model.add(Dense(1, activation='linear'))
    
    model.compile(optimizer=parameters.Choice('optmz', ['adam', 'adaDelta', 'adaGrad']),loss=parameters.Choice('loss f', ['mse', 'mae']))
    #model.compile(optimizer=Adam(parameters.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),loss='mse')
    
    return model

In [9]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(build_model,objective='val_loss',max_trials=5,executions_per_trial=3,directory='project',
                     project_name='heart-risk')

<ipython-input-9-b45e668e7ffa>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


[more info](https://keras-team.github.io/keras-tuner/)

In [10]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
#neurons layer0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_function 0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
drop_prob 1 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
#neurons layer1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_function 1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
drop_prob 2 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
optmz (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'adaDelta', 'adaGrad'], 'ordered': False}
loss f (Choic

In [11]:
tuner.search(train_data,train_target,epochs=10,validation_data=(test_data, test_target))

Trial 5 Complete [00h 00m 51s]
val_loss: 0.02368810400366783

Best val_loss So Far: 0.023663746193051338
Total elapsed time: 00h 05m 21s
INFO:tensorflow:Oracle triggered exit


In [9]:
tuner.results_summary()

Results summary
Results in project\heart-risk
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
num_layers: 6
#neurons layer0: 32
activation_function 0: relu
drop_prob 1: 0.5
#neurons layer1: 320
activation_function 1: relu
drop_prob 2: 0.2
optmz: adaDelta
#neurons layer2: 480
activation_function 2: relu
#neurons layer3: 384
activation_function 3: sigmoid
#neurons layer4: 64
activation_function 4: relu
#neurons layer5: 352
activation_function 5: sigmoid
#neurons layer6: 256
activation_function 6: relu
#neurons layer7: 480
activation_function 7: relu
#neurons layer8: 480
activation_function 8: sigmoid
#neurons layer9: 64
activation_function 9: sigmoid
#neurons layer10: 512
activation_function 10: sigmoid
#neurons layer11: 64
activation_function 11: relu
Score: 0.024247843772172928
Trial summary
Hyperparameters:
num_layers: 8
#neurons layer0: 512
activation_function 0: tanh
drop_prob 1: 0.2
#neurons layer1: 256
activation_function 1: relu
d

In [10]:
best_models = tuner.get_best_models()
print(best_models[0].summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                256       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 320)               10560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 320)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 480)               154080    
_________________________________________________________________
dropout_2 (Dropout)          (None, 480)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 384)               1